# Check GPU options

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6069248090237259449
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6984105984
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4131367650262868376
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 6913851392
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4870729057370114963
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:07:00.0, compute capability: 7.5"
xla_global_id: 2144165316
]


2022-01-04 19:53:27.739114: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-04 19:53:28.000427: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 19:53:28.000757: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 19:53:28.026911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

# Import needed libraries

In [2]:
# Import general purpose python libraries
import os
import matplotlib.pyplot as plt
from PIL import Image # For handling the images
import numpy as np
from sklearn import metrics
import pandas as pd
# Import different Keras functionalities
import tensorflow as tf
from tensorflow import keras
from keras.constraints import maxnorm
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD,Adagrad
# from keras.optimizers import SGD,Adagrad
from keras import backend as K
#from keras.applications import ResNet50,Xception,VGG16,VGG19
from sklearn.model_selection import train_test_split

from plotcm import plotcm

####
# Where to save the figures
IMAGES_PATH = "../images"
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
from utils import saveFigures,write_json,create_json,load_previous_weights

from models import VGG16, MLP_model

# Load data

In [3]:
cifar10 = keras.datasets.cifar10
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=1)

X_train = X_train/255.
X_valid = X_valid/255.
X_test = X_test/255.

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]
class_names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

print("num_training_samples: ", X_train.shape)

num_training_samples:  (40000, 32, 32, 3)


# Configuration parameters
Configuration values of different parts of the solution. You should change some of them to obtain better results.

In [4]:
from datetime import datetime
# Randomize the initial network 
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)


# Directory where to store weights of the model and results
now = datetime.now()
current_time = now.strftime("%d_%H_%M_%S")
experiment_rootdir = "./test/"+current_time+"/"
weights_path = experiment_rootdir +"weights.h5"
weights_best_path = experiment_rootdir +"weights_best.h5"
json_path = experiment_rootdir +"experiment.json"

# Create experiment directory if it does not exists
if not os.path.exists(experiment_rootdir):
    os.makedirs(experiment_rootdir)
print(experiment_rootdir)

./test/04_19_53_29/


In [5]:
# Parameters that configures the training process

# Tunable parameters
name_model = "Prueba"
initial_lr = 1e-3 

l2_reg = 0.0
dropout = 0.0

compile_parameters = { "metrics": ['accuracy'], "optimizer": SGD(learning_rate=initial_lr)}

# TODO


## Constant parameters
batch_size = 512 
epochs = 100
initial_epoch = 0 
num_classes = 10
dropout = 0.5
compile_parameters = { "metrics": ['accuracy'],
                      "optimizer": SGD(learning_rate=0.001, momentum=0.9)}
checkpoint_cb = keras.callbacks.ModelCheckpoint(weights_best_path,
save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
restore_best_weights=True)
'''
optimizer=  = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
'''

parameters = {
    "name_model":name_model,
    "compile_parameters" :str(compile_parameters) ,
    "batch_size": batch_size,
    "epochs": epochs, 
    "initial_epoch":initial_epoch, 
    "initial_lr": initial_lr,
    "num_classes": num_classes,
    "dropout": dropout

}
create_json({"parameters":parameters}, json_path)


In [6]:
def trainModel(model,compile_parameters,X_train, y_train, validation_data, epochs, batch_size, experiment_rootdir, weights_path, json_path):
    model.compile(loss='categorical_crossentropy', optimizer=compile_parameters['optimizer'], metrics=compile_parameters['metrics'])
    print(model.summary())
    history = model.fit(x= X_train, y= y_train, validation_data = validation_data, epochs = epochs, batch_size= batch_size,callbacks=[checkpoint_cb, early_stopping_cb])
    model.save_weights(weights_path)
    hist_df = pd.DataFrame(history.history) 
    data = {"train_result2":hist_df.to_dict()}
    write_json(data,json_path)
    return history

# Training 

In [ ]:
#Get model
model = MLP_model(2, num_classes=10, img_width=32, img_height=32, img_channels=3)

#load weights
load_previous_weights(model,weights_path)

#training model
val = (X_test, y_test)

history =trainModel(model,compile_parameters,X_train, y_train,val, epochs, batch_size,experiment_rootdir,weights_path,json_path)

saveFigures(experiment_rootdir, history, "accuracy", "loss")

Impossible to find weight path. Returning untrained model
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 689)               2117297   
                                                                 
 dense_1 (Dense)             (None, 344)               237360    
                                                                 
 dense_2 (Dense)             (None, 10)                3450      
                                                                 
Total params: 2,358,107
Trainable params: 2,358,107
Non-trainable params: 0
___________________________________________________________

2022-01-04 19:53:29.696252: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 19:53:29.696637: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 19:53:29.696943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 19:53:29.697227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 19:53:29.697530: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

Epoch 1/100
79/79 [==============================] - 2s 14ms/step - loss: 2.1655 - accuracy: 0.2172 - val_loss: 2.0391 - val_accuracy: 0.2916
Epoch 2/100
79/79 [==============================] - 1s 9ms/step - loss: 1.9841 - accuracy: 0.3082 - val_loss: 1.9408 - val_accuracy: 0.3268
Epoch 3/100
79/79 [==============================] - 1s 9ms/step - loss: 1.9117 - accuracy: 0.3369 - val_loss: 1.8848 - val_accuracy: 0.3477
Epoch 4/100
79/79 [==============================] - 1s 10ms/step - loss: 1.8659 - accuracy: 0.3530 - val_loss: 1.8494 - val_accuracy: 0.3629
Epoch 5/100
79/79 [==============================] - 1s 9ms/step - loss: 1.8334 - accuracy: 0.3640 - val_loss: 1.8229 - val_accuracy: 0.3680
Epoch 6/100
79/79 [==============================] - 1s 9ms/step - loss: 1.8065 - accuracy: 0.3723 - val_loss: 1.7974 - val_accuracy: 0.3796
Epoch 7/100
79/79 [==============================] - 1s 9ms/step - loss: 1.7844 - accuracy: 0.3824 - val_loss: 1.7782 - val_accuracy: 0.3855
Epoch 8/100

In [ ]:

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "batch_size": np.arange(10, 40, 10),
    "initial_lr": np.arange(0.00001, 0.0001, 0.00001),
    "dropout": np.arange(0.1, 0.9, 0.1)
}

rnd_search_cv = RandomizedSearchCV(model, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

# Testing

In [ ]:
model = VGG16(dropout, num_classes, img_width=32, img_height=32, img_channels=3)
# Load saved weights
model.compile(loss='categorical_crossentropy', optimizer=compile_parameters['optimizer'], metrics=compile_parameters['metrics'])

y_pred = np.argmax(model.predict(X_test), axis=1)
gt = np.argmax(y_test, axis=1)

# Evaluate predictions: Average accuracy and highest errors
print("-----------------------------------------------")
print("Evaluation:")
# Compute average accuracy
ave_accuracy = metrics.accuracy_score(gt, y_pred)
print('Average accuracy = ', ave_accuracy)
print("-----------------------------------------------")

# Visualize confusion matrix                                           
plotcm(experiment_rootdir, gt, y_pred,class_names, experiment_rootdir, normalize=True)
